In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/6章'
# os.chdir(path)

# ６章 機械学習のためのデータ加工をする１０本ノック



### ノック５１：データ加工の下準備をしよう

In [4]:
import os
data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
master_dir = os.path.join(data_dir, '99_master')
os.makedirs(input_dir,exist_ok=True)
os.makedirs(output_dir,exist_ok=True)
os.makedirs(master_dir,exist_ok=True)

##### ＊＊必ずデータの配置をおこなってください。＊＊

#### MacやGoogle Colaboratory等、環境によっては、ファイル取得時の順番が年月順にならないことがあります。
#### その場合、tbl_order_paths = sorted(tbl_order_paths)を入れてください。

In [5]:
import glob
tbl_order_file = os.path.join(input_dir, 'tbl_order_*.csv')
tbl_order_paths = glob.glob(tbl_order_file)
tbl_order_paths

['data/0_input/tbl_order_201910.csv',
 'data/0_input/tbl_order_201909.csv',
 'data/0_input/tbl_order_201908.csv',
 'data/0_input/tbl_order_201911.csv',
 'data/0_input/tbl_order_202002.csv',
 'data/0_input/tbl_order_201912.csv',
 'data/0_input/tbl_order_202001.csv',
 'data/0_input/tbl_order_202003.csv',
 'data/0_input/tbl_order_201907.csv',
 'data/0_input/tbl_order_201906.csv',
 'data/0_input/tbl_order_201905.csv',
 'data/0_input/tbl_order_201904.csv']

### ノック５２：データの読み込みを行い加工の方向性を検討しよう

In [6]:
import pandas as pd
m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [7]:
tbl_order_path = tbl_order_paths[0]
print(f'読み込みデータ：{tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
print(f'データ件数：{len(order_data)}')
order_data.head(3)

読み込みデータ：data/0_input/tbl_order_201910.csv
データ件数：241060


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,34908579,12,C06977721,52,37277360,2019-10-01 11:00:00,2019-10-01 11:26:00,1,2312,1
1,98345530,49,C93012586,36,5933141,2019-10-01 11:00:00,2019-10-01 11:18:00,1,939,1
2,51870569,58,C02826686,34,71448360,2019-10-01 11:00:00,2019-10-01 11:45:00,0,2312,2


### ノック５３：１か月分のデータの基本的なデータ加工を実施しよう

In [8]:
order_data = order_data.loc[order_data['store_id'] != 999]

order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'

order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,34908579,12,C06977721,52,37277360,2019-10-01 11:00:00,2019-10-01 11:26:00,1,2312,1,西葛西店,TK,東京,東京,お持ち帰り,お支払済
1,98345530,49,C93012586,36,5933141,2019-10-01 11:00:00,2019-10-01 11:18:00,1,939,1,浅草店,TK,東京,東京,お持ち帰り,お支払済
2,51870569,58,C02826686,34,71448360,2019-10-01 11:00:00,2019-10-01 11:45:00,0,2312,2,池袋店,TK,東京,東京,デリバリー,お渡し済


In [9]:
order_data.isna().sum()

order_id             0
store_id             0
customer_id          0
coupon_cd            0
sales_detail_id      0
order_accept_date    0
delivered_date       0
takeout_flag         0
total_amount         0
status               0
store_name           0
area_cd              0
wide_area            0
narrow_area          0
takeout_name         0
status_name          0
dtype: int64

### ノック５４：機械学習に使用する変数を作成しよう

In [10]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

order_data.loc[:,'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
order_data.loc[:,'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
order_data.loc[:,'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta
0,34908579,12,C06977721,52,37277360,2019-10-01 11:00:00,2019-10-01 11:26:00,1,2312,1,西葛西店,TK,東京,東京,お持ち帰り,お支払済,2019-10-01 11:00:00,2019-10-01 11:26:00,26.0
1,98345530,49,C93012586,36,5933141,2019-10-01 11:00:00,2019-10-01 11:18:00,1,939,1,浅草店,TK,東京,東京,お持ち帰り,お支払済,2019-10-01 11:00:00,2019-10-01 11:18:00,18.0
2,51870569,58,C02826686,34,71448360,2019-10-01 11:00:00,2019-10-01 11:45:00,0,2312,2,池袋店,TK,東京,東京,デリバリー,お渡し済,2019-10-01 11:00:00,2019-10-01 11:45:00,45.0


In [11]:
order_data.loc[:,'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
order_data.loc[:,'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '休日'
order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '平日'
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,...,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta,order_accept_hour,order_accept_weekday,weekday_info
0,34908579,12,C06977721,52,37277360,2019-10-01 11:00:00,2019-10-01 11:26:00,1,2312,1,...,東京,東京,お持ち帰り,お支払済,2019-10-01 11:00:00,2019-10-01 11:26:00,26.0,11,1,平日
1,98345530,49,C93012586,36,5933141,2019-10-01 11:00:00,2019-10-01 11:18:00,1,939,1,...,東京,東京,お持ち帰り,お支払済,2019-10-01 11:00:00,2019-10-01 11:18:00,18.0,11,1,平日
2,51870569,58,C02826686,34,71448360,2019-10-01 11:00:00,2019-10-01 11:45:00,0,2312,2,...,東京,東京,デリバリー,お渡し済,2019-10-01 11:00:00,2019-10-01 11:45:00,45.0,11,1,平日


### ノック５５：店舗単位に集計して変数を作成しよう

In [12]:
store_data = order_data.groupby(['store_name']).count()[['order_id']]
store_f = order_data.loc[(order_data['status_name']=="お渡し済")|
                         (order_data['status_name']=="お支払済")].groupby(['store_name']).count()[['order_id']]
store_c = order_data.loc[order_data['status_name']=="キャンセル"].groupby(['store_name']).count()[['order_id']]
store_d = order_data.loc[order_data['takeout_name']=="デリバリー"].groupby(['store_name']).count()[['order_id']]
store_t = order_data.loc[order_data['takeout_name']=="お持ち帰り"].groupby(['store_name']).count()[['order_id']]

store_weekday = order_data.loc[order_data['weekday_info']=="平日"].groupby(['store_name']).count()[['order_id']]
store_weekend = order_data.loc[order_data['weekday_info']=="休日"].groupby(['store_name']).count()[['order_id']]

In [13]:
times = order_data['order_accept_hour'].unique()
store_time = []
for time in times:
    time_tmp = order_data.loc[order_data['order_accept_hour']==time].groupby(['store_name']).count()[['order_id']]
    time_tmp.columns = [f'order_time_{time}']
    store_time.append(time_tmp)
store_time = pd.concat(store_time, axis=1)
store_time.head(3)

,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21
store_name,,,,,,,,,,,
あきる野店,109,104,113,108,96,111,90,126,112,116,104
さいたま南店,140,125,133,167,143,127,174,136,141,150,120
さいたま緑店,86,101,108,100,94,100,92,107,88,94,95


In [14]:
store_delta = order_data.loc[(order_data['status_name']!="キャンセル")].groupby(['store_name']).mean()[['delta']]
store_data.columns = ['order']
store_f.columns = ['order_fin']
store_c.columns = ['order_cancel']
store_d.columns = ['order_delivery']
store_t.columns = ['order_takeout']
store_weekday.columns = ['order_weekday']
store_weekend.columns = ['order_weekend']
store_delta.columns = ['delta_avg']
store_data = pd.concat([store_data, store_f, store_c, store_d, store_t, 
                        store_weekday, store_weekend, store_time, store_delta], axis=1)
store_data.head(3)

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg
store_name,,,,,,,,,,,,,,,,,,,
あきる野店,1189,970,219,848,341,880,309,109,104,113,108,96,111,90,126,112,116,104,34.569072
さいたま南店,1556,1278,278,1155,401,1155,401,140,125,133,167,143,127,174,136,141,150,120,34.309077
さいたま緑店,1065,852,213,789,276,789,276,86,101,108,100,94,100,92,107,88,94,95,34.546948


### ノック５６：データの加工と店舗別集計を関数で実行しよう

In [15]:
def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 999]
    order_data = pd.merge(order_data, m_store, on='store_id', how='left')
    order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
    order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
    order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
    order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'
    
    order_data.loc[:,'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
    order_data.loc[:,'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
    order_data.loc[:,'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
    order_data.loc[:,'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
    order_data.loc[:,'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '休日'
    order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '平日'
    
    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[(order_data['status_name']=="お渡し済")|
                             (order_data['status_name']=="お支払済")].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[order_data['status_name']=="キャンセル"].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[order_data['takeout_name']=="デリバリー"].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[order_data['takeout_name']=="お持ち帰り"].groupby(['store_name']).count()[['order_id']]
    store_weekday = order_data.loc[order_data['weekday_info']=="平日"].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[order_data['weekday_info']=="休日"].groupby(['store_name']).count()[['order_id']]
    times = order_data['order_accept_hour'].unique()
    store_time = []
    for time in times:
        time_tmp = order_data.loc[order_data['order_accept_hour']==time].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f'order_time_{time}']
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis=1)
    store_delta = order_data.loc[order_data['status_name']!="キャンセル"].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']
    store_delta.columns = ['delta_avg']
    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_data = pd.concat([store_data, store_f, store_c, store_d, store_t, 
                        store_weekday, store_weekend, store_time, store_delta], axis=1)
    return store_data

In [16]:
tbl_order_path = tbl_order_paths[0]
print(f'読み込みデータ：{tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
store_data = data_processing(order_data)
store_data.head(3)

読み込みデータ：data/0_input/tbl_order_201910.csv


,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg
store_name,,,,,,,,,,,,,,,,,,,
あきる野店,1189,970,219,848,341,880,309,109,104,113,108,96,111,90,126,112,116,104,34.569072
さいたま南店,1556,1278,278,1155,401,1155,401,140,125,133,167,143,127,174,136,141,150,120,34.309077
さいたま緑店,1065,852,213,789,276,789,276,86,101,108,100,94,100,92,107,88,94,95,34.546948


### ノック５７：全データの読み込みとデータ加工をやってみよう

In [17]:
store_all = []
for tbl_order_path in tbl_order_paths:
    print(f'読み込みデータ：{tbl_order_path}')
    tg_ym = tbl_order_path.split('_')[-1][:6]
    order_data = pd.read_csv(tbl_order_path)
    store_data = data_processing(order_data)
    store_data.loc[:,'year_month'] = tg_ym
    store_data.reset_index(drop=False, inplace=True)
    store_all.append(store_data)
store_all = pd.concat(store_all, ignore_index=True)
display(store_all.head(3))
display(store_all.tail(3))
store_monthly_name = 'store_monthly_data.csv'
store_all.to_csv(os.path.join(output_dir, store_monthly_name), index=False)

読み込みデータ：data/0_input/tbl_order_201910.csv
読み込みデータ：data/0_input/tbl_order_201909.csv
読み込みデータ：data/0_input/tbl_order_201908.csv
読み込みデータ：data/0_input/tbl_order_201911.csv
読み込みデータ：data/0_input/tbl_order_202002.csv
読み込みデータ：data/0_input/tbl_order_201912.csv
読み込みデータ：data/0_input/tbl_order_202001.csv
読み込みデータ：data/0_input/tbl_order_202003.csv
読み込みデータ：data/0_input/tbl_order_201907.csv
読み込みデータ：data/0_input/tbl_order_201906.csv
読み込みデータ：data/0_input/tbl_order_201905.csv
読み込みデータ：data/0_input/tbl_order_201904.csv


,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1189,970,219,848,341,880,309,109,104,...,108,96,111,90,126,112,116,104,34.569072,201910
1,さいたま南店,1556,1278,278,1155,401,1155,401,140,125,...,167,143,127,174,136,141,150,120,34.309077,201910
2,さいたま緑店,1065,852,213,789,276,789,276,86,101,...,100,94,100,92,107,88,94,95,34.546948,201910


,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
2337,鴻巣店,1108,882,226,805,303,811,297,108,89,...,102,110,97,96,100,97,105,94,34.100907,201904
2338,鶴見店,1474,1226,248,1072,402,1081,393,124,146,...,136,133,148,120,123,133,147,139,34.202284,201904
2339,麻生店,1353,1093,260,1010,343,991,362,116,124,...,124,125,99,119,117,124,136,144,34.234218,201904


### ノック５８：目的変数を作成しよう

In [18]:
y = store_all[['store_name', 'year_month','order_weekday', 'order_weekend']].copy()
y.loc[:,'one_month_ago'] = pd.to_datetime(y['year_month'], format='%Y%m')
from dateutil.relativedelta import relativedelta
y.loc[:,'one_month_ago'] = y['one_month_ago'].map(lambda x: x - relativedelta(months=1))
y.loc[:,'one_month_ago'] = y['one_month_ago'].dt.strftime('%Y%m')
y.head(3)

,store_name,year_month,order_weekday,order_weekend,one_month_ago
0,あきる野店,201910,880,309,201909
1,さいたま南店,201910,1155,401,201909
2,さいたま緑店,201910,789,276,201909


In [19]:
y_one_month_ago = y.copy()
y_one_month_ago.rename(columns={'order_weekday':'order_weekday_one_month_ago', 
                                'order_weekend':'order_weekend_one_month_ago',
                                'year_month':'year_month_for_join'}, inplace=True)
y = pd.merge(y, y_one_month_ago[['store_name', 'year_month_for_join', 
                                 'order_weekday_one_month_ago', 'order_weekend_one_month_ago']], 
                                 left_on=['store_name', 'one_month_ago'], 
                                 right_on=['store_name','year_month_for_join'], how='left')
y.loc[y['store_name']=='あきる野店']

,store_name,year_month,order_weekday,order_weekend,one_month_ago,year_month_for_join,order_weekday_one_month_ago,order_weekend_one_month_ago
0,あきる野店,201910,880,309,201909,201909,802.0,347.0
195,あきる野店,201909,802,347,201908,201908,835.0,343.0
390,あきる野店,201908,835,343,201907,201907,882.0,308.0
585,あきる野店,201911,796,341,201910,201910,880.0,309.0
780,あきる野店,202002,762,344,202001,202001,881.0,305.0
975,あきる野店,201912,844,345,201911,201911,796.0,341.0
1170,あきる野店,202001,881,305,201912,201912,844.0,345.0
1365,あきる野店,202003,839,347,202002,202002,762.0,344.0
1560,あきる野店,201907,882,308,201906,201906,764.0,384.0
1755,あきる野店,201906,764,384,201905,201905,883.0,302.0


### ノック５９：説明変数と目的変数を紐づけて機械学習用のデータを仕上げよう

In [20]:
y

,store_name,year_month,order_weekday,order_weekend,one_month_ago,year_month_for_join,order_weekday_one_month_ago,order_weekend_one_month_ago
0,あきる野店,201910,880,309,201909,201909,802.0,347.0
1,さいたま南店,201910,1155,401,201909,201909,1050.0,453.0
2,さいたま緑店,201910,789,276,201909,201909,722.0,307.0
3,さいたま西店,201910,898,314,201909,201909,822.0,355.0
4,つくば店,201910,967,337,201909,201909,877.0,378.0
...,...,...,...,...,...,...,...,...
2335,高津店,201904,818,299,201903,NaN,NaN,NaN
2336,高田馬場店,201904,774,280,201903,NaN,NaN,NaN
2337,鴻巣店,201904,811,297,201903,NaN,NaN,NaN
2338,鶴見店,201904,1081,393,201903,NaN,NaN,NaN


In [21]:
# 謎の記述y_weekday,y_weekendが読み込めなかったため修正
y.rename(columns={'year_month':'target_year_month'},inplace=True)
y = y[['store_name','target_year_month', 'one_month_ago', 'order_weekday', 'order_weekend']].copy()
ml_data = pd.merge(y, store_all, left_on=['store_name','one_month_ago'],
                   right_on=['store_name','year_month'], how='left')
ml_data.head()

,store_name,target_year_month,one_month_ago,order_weekday_x,order_weekend_x,order,order_fin,order_cancel,order_delivery,order_takeout,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,201910,201909,880,309,1149.0,933.0,216.0,855.0,294.0,...,117.0,110.0,105.0,103.0,109.0,109.0,96.0,106.0,34.174705,201909
1,さいたま南店,201910,201909,1155,401,1503.0,1260.0,243.0,1115.0,388.0,...,145.0,129.0,142.0,128.0,134.0,136.0,127.0,145.0,33.911111,201909
2,さいたま緑店,201910,201909,789,276,1029.0,815.0,214.0,745.0,284.0,...,91.0,89.0,89.0,111.0,88.0,81.0,108.0,107.0,33.952147,201909
3,さいたま西店,201910,201909,898,314,1177.0,966.0,211.0,868.0,309.0,...,102.0,86.0,103.0,104.0,111.0,92.0,118.0,125.0,34.153209,201909
4,つくば店,201910,201909,967,337,1255.0,1023.0,232.0,933.0,322.0,...,114.0,113.0,94.0,123.0,108.0,117.0,100.0,118.0,33.785924,201909


In [22]:
del ml_data["target_year_month"]
del ml_data["one_month_ago"]
ml_data.head()

,store_name,order_weekday_x,order_weekend_x,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday_y,order_weekend_y,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,880,309,1149.0,933.0,216.0,855.0,294.0,802.0,347.0,...,117.0,110.0,105.0,103.0,109.0,109.0,96.0,106.0,34.174705,201909
1,さいたま南店,1155,401,1503.0,1260.0,243.0,1115.0,388.0,1050.0,453.0,...,145.0,129.0,142.0,128.0,134.0,136.0,127.0,145.0,33.911111,201909
2,さいたま緑店,789,276,1029.0,815.0,214.0,745.0,284.0,722.0,307.0,...,91.0,89.0,89.0,111.0,88.0,81.0,108.0,107.0,33.952147,201909
3,さいたま西店,898,314,1177.0,966.0,211.0,868.0,309.0,822.0,355.0,...,102.0,86.0,103.0,104.0,111.0,92.0,118.0,125.0,34.153209,201909
4,つくば店,967,337,1255.0,1023.0,232.0,933.0,322.0,877.0,378.0,...,114.0,113.0,94.0,123.0,108.0,117.0,100.0,118.0,33.785924,201909


### ノック６０：機械学習用データの確認を行い出力しよう

In [29]:
# 全て0じゃない時点でおかしい
ml_data.isna().sum()

store_name           0
order_weekday_x      0
order_weekend_x      0
order              195
order_fin          195
order_cancel       195
order_delivery     195
order_takeout      195
order_weekday_y    195
order_weekend_y    195
order_time_11      195
order_time_12      195
order_time_13      195
order_time_14      195
order_time_15      195
order_time_16      195
order_time_17      195
order_time_18      195
order_time_19      195
order_time_20      195
order_time_21      195
delta_avg          195
year_month         195
dtype: int64

In [27]:
display(ml_data.groupby('order_weekday_y').count()[['store_name']])
display(ml_data.groupby('order_weekend_y').count()[['store_name']])

,store_name
order_weekday_y,
464.0,1
469.0,1
488.0,1
489.0,1
490.0,1
...,...
1927.0,1
1929.0,1
2003.0,1


,store_name
order_weekend_y,
188.0,1
189.0,1
190.0,2
191.0,1
195.0,1
...,...
785.0,1
788.0,1
789.0,1


In [28]:
ml_data.to_csv(os.path.join(output_dir, 'ml_base_data.csv'), index=False)